### The following section includes sourced code to generate a graph coloring problem.

In [4]:
################################################################
##
## GRAPH COLORING PROBLEM GENERATOR
##
## Generates a planar graph.
##
################################################################

import random

class Point:
    ID_COUNT = 0
    def __init__(self, x, y):
        self.id = Point.ID_COUNT
        Point.ID_COUNT += 1
        self.x = x
        self.y = y

    def transform(self, xt, yt):
        return (xt(self.x), yt(self.y))

    def dist(self, them):
        return abs(self.x - them.x) + abs(self.y - them.y)

    def __repr__(self):
        return "p({:.4f}, {:.4f})".format(self.x, self.y)

## From http://bryceboe.com/2006/10/23/line-segment-intersection-algorithm/
def _ccw(A,B,C):
    return (C.y-A.y)*(B.x-A.x) > (B.y-A.y)*(C.x-A.x)

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    @property
    def endpoints(self):
        return [self.p1, self.p2]

    def transform(self, xt = lambda x: x, yt = lambda y: y):
        start = self.p1.transform(xt, yt)
        end = self.p2.transform(xt, yt)
        return (start, end)

    def intersects(self, them):
        A = self.p1
        B = self.p2
        C = them.p1
        D = them.p2
        return _ccw(A,C,D) != _ccw(B,C,D) and _ccw(A,B,C) != _ccw(A,B,D)

    def __repr__(self):
        return "[{} -> {}]".format(self.p1, self.p2)

def _random_point():
    return Point(random.uniform(-10, 10), random.uniform(-10, 10))

def _find_line(x, lines, pairs):
    for i, x1 in enumerate(x[:-1]):
        shortest_items = sorted([(x2, x1.dist(x2)) for x2 in x[i+1:]], key=lambda item: item[1])
        for x2, _ in shortest_items:
            l1 = Line(x1, x2)
            if not (x1, x2) in pairs and not _line_intersects(l1, lines):
                return l1
    return None

def _line_intersects(l1, lines):
    for l2 in lines:
        if l1.p1 in l2.endpoints or l1.p2 in l2.endpoints:
            continue
        if l1.intersects(l2): return True
    return False

def gen(num_points=100):
    x = [_random_point() for _ in range(num_points)]
    lines = set([])
    pairs = set([])
    while True:
        random.shuffle(x)
        line = _find_line(x, lines, pairs)
        if line:
            pairs.add((line.p1, line.p2))
            lines.add(line)
        else:
            break
    return (x, lines)

def draw(x, lines):
    try:
        response = input("Would you like to view the network? [Y/n] ")
        if response.lower() == 'n':
            return
        import pygame
    except ImportError:
        print("Please install pygame ('pip3 install pygame') to view the network.")
        return
    pygame.init()
    screen = pygame.display.set_mode((450, 450))
    for line in lines:
        (start, end) = line.transform(xt=lambda x: 25 + int(x * 20 + 200),
                                      yt=lambda y: 25 + int(y * 20 + 200))
        pygame.draw.line(screen, (255, 255, 255), start, end)
    print("Press ESC to exit and save the file.")
    while 1:
        # From https://stackoverflow.com/a/7055453
        for event in pygame.event.get():
           if event.type == pygame.QUIT:
               return
           elif event.type == pygame.KEYDOWN:
               if event.key == pygame.K_ESCAPE:
                   pygame.quit()
                   return
        pygame.display.flip()

In [5]:
import argparse, json

num_points = 100
output_file = "gcp.json"

print("Generating a planar graph with {} points...".format(num_points))
(x, lines) = gen(num_points=num_points)
draw(x, lines)

print("Writing to '{}'...".format(output_file))
with open(output_file, 'w') as f:
    f.write(json.dumps({
        'num_points': len(x),
        'points': { p.id: (p.x, p.y) for p in x },
        'edges': [ (line.p1.id, line.p2.id) for line in lines ]
    }, indent=2))
print("""Done! You can now import the results into a Python script with the code:

import json

with open('{}', \'r\') as f:
    data = json.load(f)
""".format(output_file))

Generating a planar graph with 100 points...
Writing to 'gcp.json'...
Done! You can now import the results into a Python script with the code:

import json

with open('gcp.json', 'r') as f:
    data = json.load(f)



In [6]:
import json

with open('gcp.json', 'r') as f:
    data = json.load(f)
    print(data)

{'num_points': 100, 'points': {'14': [-0.971683157100383, -8.356551419559363], '99': [2.4516731074779905, 2.156026371034411], '36': [3.34541070258741, 2.0359634288755153], '22': [9.904950890101237, -5.933294771202091], '6': [-0.3666310444739338, -7.565912140040407], '41': [-5.255434806601809, 8.141148884268944], '56': [7.5734233571164395, 6.294672493489571], '84': [1.0329709707682415, 4.359185731796309], '95': [4.148988308827551, 8.99154430170056], '33': [-9.050350233338534, -5.6998505907087305], '42': [-5.215291162229494, -9.582022238575274], '4': [8.347289994164871, -7.369848379280239], '29': [9.375680680426022, 4.75779054998528], '59': [5.1313459652020335, 7.062716296129249], '55': [9.779453943484466, -7.081093996678749], '65': [-9.514531414762516, -4.481510638673194], '68': [5.820788789516822, 6.299545414697654], '16': [6.6871137558747655, -6.05880396206814], '82': [5.890391708627387, -5.192341253315426], '44': [-9.923068122038812, 1.0645936199280044], '85': [-8.292574021660652, 7.

### The following section includes sourced code to generate a Sudoku problem.

In [7]:
################################################################
## CLI Wrapper for a Sudoku generator
##  by Chad Crawford, using code from Gareth Rees
################################################################

import random
from functools import *

## From Gareth Rees at https://codereview.stackexchange.com/a/88866
def make_board(m: int=3) -> list[list[int]]:
    """Return a random filled m**2 x m**2 Sudoku board."""
    n = m**2
    board = [[None for _ in range(n)] for _ in range(n)]

    def search(c: int=0) -> list[list[int]]:
        "Recursively search for a solution starting at position c."
        i, j = divmod(c, n)
        i0, j0 = i - i % m, j - j % m # Origin of mxm block
        numbers = list(range(1, n + 1))
        random.shuffle(numbers)
        for x in numbers:
            if (x not in board[i]                     # row
                and all(row[j] != x for row in board) # column
                and all(x not in row[j0:j0+m]         # block
                        for row in board[i0:i])):
                board[i][j] = x
                if c + 1 >= n**2 or search(c + 1):
                    return board
        else:
            # No number is valid in this cell: backtrack and try again.
            board[i][j] = None
            return None

    return search()

In [8]:
import json
import argparse

num_missing = 10

parser = argparse.ArgumentParser(description='Generates random Sudoku board configurations. Represented as a Python list of lists.')
parser.add_argument('num_missing', type=int, help='Number of empty slots in the board. Must be less than 81.')
parser.add_argument('--output_file', type=str, default='sudoku.json', help='File to write the board configuration to.')
parser.add_argument('--block_size', type=int, default=3, help='Size of the blocks on the Sudoku board.')
args = parser.parse_args([f'{num_missing}'])

board = make_board(args.block_size)

ms = int(pow(args.block_size, 2))
indices = random.sample(list(range(ms * ms)), args.num_missing)
for index in indices:
    r = int(index / ms)
    c = int(index % ms)
    board[r][c] = 0

## Write to file
with open(args.output_file, 'w') as f:
    f.write(json.dumps(board))

print(f"Board written to '{args.output_file}'.")
print("""Done! You can now import the results into a Python script with the code:

import json

with open('{}', \'r\') as f:
    data = json.load(f)
""".format(args.output_file))

Board written to 'sudoku.json'.
Done! You can now import the results into a Python script with the code:

import json

with open('sudoku.json', 'r') as f:
    data = json.load(f)



In [9]:
import json
output_file = 'sudoku.json'
with open(output_file, 'r') as f:
    board = json.load(f)

print(board)

[[7, 8, 6, 9, 5, 3, 1, 4, 2], [2, 9, 4, 8, 1, 0, 5, 6, 3], [1, 5, 3, 4, 2, 0, 9, 7, 8], [8, 7, 9, 5, 6, 2, 4, 3, 1], [5, 6, 2, 0, 4, 1, 8, 9, 7], [4, 3, 1, 0, 8, 9, 2, 5, 6], [3, 1, 0, 2, 9, 5, 6, 8, 4], [9, 0, 8, 6, 0, 4, 0, 1, 5], [6, 4, 5, 0, 7, 0, 3, 2, 9]]


### The following section includes my own implementation of solving a constraint satisfaction problem.

In [2]:
# Before we get into solving a constraint satisfaction problem, we need to define one.
from enum import Enum

class ArcConsistency(Enum):
    AC1 = 1
    AC2 = 2
    AC3 = 3

class Graph:
    """
    The graph class will be responsible for remembering connections between nodes and maintaining the variables available for each node.
    In addition, when a node is assigned a value, this class is responsible for maintaining the:
     - Brute force backtracking variable options for each node (AC1)
     - Forward checking variable options for each node (AC2)
     - Three-way arc consistency variable options for each node (AC3)
    This class is not going to maintain all three such options at once, but it will have all three options implemented and will perform whichever algorithm is specified in its initialization.
    """

    class __Node:
        """
        The underlying node class for a Graph
        """
        def __init__(self, num_classes: int):
            """
            A node has a set of neighbor nodes and a set of variable options it can be assigned.
            It does not actually have to know its id or its currently assigned value - the Graph class will take care of the book-keeping.
            """
            self.connections = set()
            self.options = set()
            for i in range(num_classes):
                self.options.add(i)
        
        def get_degree(self) -> int:
            """
            Return the number of neighbors this node has
            """
            return len(self.connections)
        
        def get_num_options(self) -> int:
            """
            Return the number of variable options this node has
            """
            return len(self.options)
        
        def remove(self, option: int):
            """
            Remove this option from the node's set of options
            """
            self.options.remove(option)
        
        def add_neighbor(self, other_id: int):
            """
            Add a neighbor to this node's set of neighbors
            """
            self.connections.add(other_id)


    def __init__(self, num_classes: int, consistency: ArcConsistency):
        """
        When initializing a Graph object, all we need to know is the number of possible values that can be assigned to each of this graph's nodes.
        """
        self.__num_classes = num_classes
        self.__consistency = ArcConsistency
        self.__num_assigned = 0
        self.__graph = {}
        self.__values = {}
    
    def __add_node(self, id: int):
        """
        Helper method to add a node to our underlying graph
        """
        self.__graph[id] = Graph.__Node(num_classes=self.__num_classes, id=id)
        self.__values[id] = None

    def connect(self, first: int, second: int):
        """
        Create the two nodes if they have not already been created, and connect those two nodes to each other
        """
        if first not in self.__graph.keys():
            self.__add_node(id=first)
        if second not in self.__graph.keys():
            self.__add_node(id=second)
        self.__graph[first].add_connection(second)
        self.__graph[second].add_connection(first)

    def is_solved(self) -> bool:
        """
        Return if this graph has all of its variables assigned
        """
        return self.__num_assigned == len(self.__graph)

    def assign(self, id: int)->bool:
        """
        Assign any value that works to a given node, and update all other nodes' available values according to the arc consistency of this graph.
        If no value works for the assignment, return False
        """
        for option in self.__graph[id].options:
            # Try assigning this value to the variable
            self.__values[id] = option
            if self.__update_available(id=id):
                # As far as we know according to our arc consistency, we haven't backed ourselves into an unsolvable assignment set yet...
                self.__num_assigned += 1
                return True
        
        return False
    
    def __update_available(self, id: int) -> bool:
        """
        Given a node that has just been assigned a value, update all neighbors' available values according to our arc consistency, and return False if any problems occur.
        """
        if self.__consistency == ArcConsistency.AC1:
            # We're not doing any kind of forward checking and we only run into a problem once we hit a node with no assignment
            return True
        elif self.__consistency == ArcConsistency.AC2:
            # Forward checking
            pass
        else:
            # AC3 consistency
            pass

In [ ]:
class CSPSolver:
    pass